In [4]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import pydicom
import cv2

# Define your data directories and parameters
train_data_dir = r'D:\Research\Neurodegenerative Diseases\Images\Task1\Task1-Images\MRI\Training'
test_data_dir = r'D:\Research\Neurodegenerative Diseases\Images\Task1\Task1-Images\MRI\Testing'
batch_size = 32
image_size = (128, 128) 
num_epochs = 2
num_classes = 5  # CN, AD, MCI, EMCI, LMCI

# Function to load and preprocess images and labels
def load_and_preprocess_images_and_labels(directory):
    image_data = []
    labels = []

    for filename in os.listdir(directory):
        if filename.endswith(".dcm"):
            dcm_path = os.path.join(directory, filename)
            try:
                dcm_data = pydicom.dcmread(dcm_path)
                img = dcm_data.pixel_array.astype(np.float32) / 255.0
                img = cv2.resize(img, image_size)
                image_data.append(img)
                
                # Parse the filename to extract class name
                filename_parts = filename.split('_')
                class_name = filename_parts[-1].split('.')[0]
                
                # Map class names to class labels (you can customize this)
                class_mapping = {'CN': 0, 'AD': 1, 'MCI': 2, 'EMCI': 3, 'LMCI': 4}
                label = class_mapping.get(class_name, -1)  # Assign labels as needed
                
                if label != -1:
                    labels.append(label)
                else:
                    print(f"Unknown class name: {class_name}")
            except Exception as e:
                print(f"Error processing file {dcm_path}: {str(e)}")

    return np.array(image_data), np.array(labels)

# Load and preprocess training and testing data
train_images, train_labels = load_and_preprocess_images_and_labels(train_data_dir)
test_images, test_labels = load_and_preprocess_images_and_labels(test_data_dir)

# Expand grayscale images to three channels (RGB)
train_images = np.repeat(train_images[:, :, :, np.newaxis], 3, axis=-1)
test_images = np.repeat(test_images[:, :, :, np.newaxis], 3, axis=-1)

# Convert labels to one-hot encoding
train_labels = tf.keras.utils.to_categorical(train_labels, num_classes)
test_labels = tf.keras.utils.to_categorical(test_labels, num_classes)

# Rest of your code remains the same...

# Create a data generator for training images
train_data_generator = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Load the ResNet50 model
base_model = ResNet50(weights='imagenet', input_shape=(128, 128, 3), include_top=False)

# Modify the last layer of your model
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(num_classes, activation='softmax')(x)  # Use softmax activation

model = Model(inputs=base_model.input, outputs=predictions)

# Freeze the layers of the ResNet50 model
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# Apply oversampling only if the class distribution is imbalanced
class_distribution = Counter(np.argmax(train_labels, axis=1))
max_class_samples = max(class_distribution.values())

if len(class_distribution) > 1 and max_class_samples < len(train_labels) / len(class_distribution):
    oversampler = RandomOverSampler(random_state=42)
    train_images_resampled, train_labels_resampled = oversampler.fit_resample(
        train_images.reshape(-1, image_size[0]*image_size[1]*3),
        train_labels.argmax(axis=1)  # Use integer labels
    )

    # Convert integer labels to one-hot encoding
    train_labels_resampled = tf.keras.utils.to_categorical(train_labels_resampled, num_classes)

    # Train the model using the data generator
    for epoch in range(num_epochs):
        print(f"Epoch {epoch + 1}/{num_epochs}")
        history = model.fit(
            train_data_generator.flow(train_images_resampled, train_labels_resampled, batch_size=batch_size),
            steps_per_epoch=len(train_images_resampled) // batch_size,
            epochs=1,  # Train for one epoch at a time
        )
else:
    # Train the model using the original unbalanced data
    for epoch in range(num_epochs):
        print(f"Epoch {epoch + 1}/{num_epochs}")
        history = model.fit(
            train_data_generator.flow(train_images, train_labels, batch_size=batch_size),
            steps_per_epoch=len(train_images) // batch_size,
            epochs=1,  # Train for one epoch at a time
        )

# Extract features from the Global Average Pooling layer
feature_extractor = Model(inputs=model.input, outputs=model.layers[-2].output)
train_features = feature_extractor.predict(train_images)
test_features = feature_extractor.predict(test_images)

# Flatten the extracted features
train_features_flat = train_features.reshape(train_features.shape[0], -1)
test_features_flat = test_features.reshape(test_features.shape[0], -1)

# Apply Logistic Regression classifier
classifier = LogisticRegression(max_iter=1000)
classifier.fit(train_features_flat, train_labels.argmax(axis=1))
test_predictions = classifier.predict(test_features_flat)

# Calculate and print test accuracy
test_accuracy = accuracy_score(np.argmax(test_labels, axis=1), test_predictions)
print(f"Test Accuracy (Logistic Regression): {test_accuracy * 100:.2f}%")


Epoch 1/2
421/421 [==============================] - 582s 1s/step - loss: 1.4721 - accuracy: 0.3085
Epoch 2/2
106/106 [==============================] - 128s 1s/step
Test Accuracy (Logistic Regression): 40.72%


c:\Python311\Lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
